In [2]:
pip install deepctr

Defaulting to user installation because normal site-packages is not writeable
     |████████████████████████████████| 110 kB 185 kB/s eta 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [1]:
import pandas as pd
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

from deepctr.models import DeepFM
from deepctr.feature_column import SparseFeat, get_feature_names

In [2]:
data = pd.read_csv("movielens_sample.txt")
sparse_features = ["movie_id", "user_id", "gender", "age", "occupation", "zip"]
target = ['rating']

# 对特征标签进行编码
for feature in sparse_features:
    lbe = LabelEncoder()
    data[feature] = lbe.fit_transform(data[feature])
# 计算每个特征中的 不同特征值的个数
fixlen_feature_columns = [SparseFeat(feature, data[feature].nunique()) for feature in sparse_features]
linear_feature_columns = fixlen_feature_columns
dnn_feature_columns = fixlen_feature_columns
feature_names = get_feature_names(linear_feature_columns + dnn_feature_columns)

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


In [3]:
# 将数据集切分成训练集和测试集
train, test = train_test_split(data, test_size=0.2)
train_model_input = {name:train[name].values for name in feature_names}
test_model_input = {name:test[name].values for name in feature_names}

In [4]:
# 使用DeepFM进行训练
model = DeepFM(linear_feature_columns, dnn_feature_columns, task='regression')
model.compile("adam", "mse", metrics=['mse'], )
history = model.fit(train_model_input, train[target].values, batch_size=256, epochs=50, verbose=True, validation_split=0.2, )


Epoch 1/50


/opt/conda/lib/python3.7/site-packages/tensorflow/python/framework/indexed_slices.py:434: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


1/1 [==============================] - 1s 873ms/step - loss: 14.8436 - mse: 14.8436 - val_loss: 13.1081 - val_mse: 13.1081
Epoch 2/50
1/1 [==============================] - 0s 154ms/step - loss: 14.7025 - mse: 14.7025 - val_loss: 12.9922 - val_mse: 12.9922
Epoch 3/50
1/1 [==============================] - 0s 86ms/step - loss: 14.5524 - mse: 14.5524 - val_loss: 12.8684 - val_mse: 12.8684
Epoch 4/50
1/1 [==============================] - 0s 157ms/step - loss: 14.3920 - mse: 14.3920 - val_loss: 12.7370 - val_mse: 12.7370
Epoch 5/50
1/1 [==============================] - 0s 88ms/step - loss: 14.2219 - mse: 14.2219 - val_loss: 12.5973 - val_mse: 12.5973
Epoch 6/50
1/1 [==============================] - 0s 157ms/step - loss: 14.0414 - mse: 14.0414 - val_loss: 12.4490 - val_mse: 12.4490
Epoch 7/50
1/1 [==============================] - 0s 154ms/step - loss: 13.8495 - mse: 13.8495 - val_loss: 12.2920 - val_mse: 12.2920
Epoch 8/50
1/1 [==============================] - 0s 162ms/step - loss: 13.

In [5]:
# 使用DeepFM进行预测
pred_ans = model.predict(test_model_input, batch_size=256)
# 输出RMSE或MSE
mse = round(mean_squared_error(test[target].values, pred_ans), 4)
rmse = mse ** 0.5
print("test RMSE", rmse)

test RMSE 0.9496841580230767
